In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from math import log, sqrt, exp, pi, cos
from scipy import stats as st
import seaborn as sns
sns.set(color_codes=True)

sns.set_palette(sns.color_palette("Set2"))
sns.set(rc={'figure.figsize':(10,8)}, style="white", color_codes=True)


import warnings
warnings.filterwarnings("ignore")

In [4]:
m = 2**32 
a = 1013904223
c = 1664525
U0 = int((94335 + 89397 + 95172 + 93965)/4)

numsGen = {}
numsGen[0] = U0

def UGen(n):
    if (n in numsGen) :
        return numsGen[n]
    else:
        numsGen[n] = (a*UGen(n-1) + c) % m
        return numsGen[n]

# Primeros 10 Números Aleatorios
for i in range(1,11):
    print(UGen(i))

2356271436
1581526337
1615389740
1185142369
2500986636
1168271233
3037413868
2185355425
535991500
3724820929


### Ejercicio 5
Aplicar un gap test al generador congruencial lineal implementado en el ejercicio 1 utilizando el intervalo [0,2 -0,5]. Analizar el resultado obtenido, e indicar si la distribución de probabilidades pasa o no el test. Considerar un nivel de significación del 1%. 

---

**Gap Test**: Siendo $U_{i}$ un número aleatorio con **distribución uniforme** en un rango $[0,1]$ y $0 \leq \alpha < \beta \leq 1$. Este test consiste en contar la cantidad de números aleatorios generados de forma consecutiva entre la generación de un número que pertenece al intervalo $[\alpha, \beta]$ y el próximo número generado que vuelve a pertenecer al intervalo.

Entendemos como gap al espacio entre dos apariciones consecutivas de números pertenecientes al intervalo $[\alpha, \beta]$.

La probabilidad teórica de apariciones de tamaño de gaps es:

$P(tamanioDeGap = 0) = p = \beta - \alpha $ 

$P(tamanioDeGap = 1) = p(1-p)$

$...$

$P(tamanioDeGap = t) = p(1-p)ˆt$

La distribución de frecuencia teórica es:

$ P(tamanioDeGap \leq t) = F(t) = p\sum_{n=1}^{t} (1-p)^{n} = 1-(1-p)^{t+1}$


---
Tomamos:

$\alpha=0.2,\qquad  \beta=0.5$

y queremos analizar un total de 100 gaps.

In [5]:
def perteneceAlIntervalo(num, alfa, beta):
    return (num>=alfa and num<=beta)

def buscarGaps(n, alfa, beta):
    tamanioGaps = 0
    gaps = list()
    i = 0

    while (len(gaps) <= n):
        
        number = UGen(i) / m
        
        if (not perteneceAlIntervalo(number, alfa, beta)):
            tamanioGaps += 1

        if (perteneceAlIntervalo(number, alfa, beta)):
            gaps.append(tamanioGaps)
            tamanioGaps = 0

        i += 1

    gaps = gaps[1:] 
    return gaps

gaps = buscarGaps(100, 0.2, 0.5)
tamanioGaps, frecuencia = np.unique(gaps, return_counts = True)

print("Gaps encontrados: " +  str(len(gaps)))
print("Tamaños de gaps encontrados: "+ str(len(tamanioGaps)))
print(tamanioGaps)
print("Frecuencia: ")
print(frecuencia)


Gaps encontrados: 100
Tamaños de gaps encontrados: 11
[ 0  1  2  3  4  5  6  8  9 10 11]
Frecuencia: 
[35 19 17  6  4  9  1  2  2  2  3]


Para 100 gaps encontrados:

| Tamaño de gap | Apariciones  | Probabilidad Obtenida | Probabilidad Teórica |

| 0 | 35| 0.35|0.3  |

| 1 | 29| 0.19|0.21 |

| 2 | 14| 0.17|0.147|

| 3 | 6 | 0.06|0.103|

| 4 | 4 | 0.04|0.072|

| 5 | 9 | 0.09|0.050|

| 6 | 1 | 0.01|0.035|

| 8 | 2 | 0.02|0.017|

| 9 | 2 | 0.02|0.012|

| 10| 2 | 0.02|0.008|

| 11| 3 | 0.03|0.006|

Ahora realizamos el **Test Estadístico $Chi^{2}$** para los datos obtenidos en el gap test:

Primero medimos la dispersión de las ocurrencias observadas $N_{i}$ respecto de las esperadas ( $np_{i}$ ):

<h3 align="center">$
D^{2} = \sum_{k=1}^{K-1}\frac {(N_{i} - np_{i})^{2}}{np_{i}}
$</h3> 

In [6]:
d2 = 0
tamanioFrecuencia = dict(zip(tamanioGaps, frecuencia)) 
pi = {0 : 0.3,
      1 : 0.21,
      2 : 0.147,
      3 : 0.103,
      4 : 0.072,
      5 : 0.050,
      6 : 0.035,
      8 : 0.017,
      9 : 0.012,
      10 : 0.008,
      11 : 0.006}
n = 100

for key in tamanioFrecuencia:
    d2 += ((tamanioFrecuencia[key] - n*(pi[key]))**2)/(n*(pi[key]))

print('D2 = ' + str(d2))

D2 = 21.573030118196144


Encontramos que nuestro $D^{2} = 21.573030118196144$  .

Con un nivel de significación del $1\%$, $\alpha = 0.01$, y una cantidad de diferentes valores k, $k=11$, buscamos en la tabla $Chi^{2}$ guiándonos por  $\alpha = 0.01$ y grados de libertad  $v = k-1 = 10 $,  tenemos  $t = 2,5$.

Como $D^{2} > t$, entonces **rechazamos la hipótesis** diciendo que los datos no responden a una distribución uniforme, considerando un error del $1\%$.